<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Black-Litterman (1992)

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Formulating Views

Topics of interest include:

* real data
 * benchmark returns
 * covariance matrix
* views
 * securities affected
 * impact on expected return(s)
 * variance of expectation
* blended expected returns
* blended covariance matrix
* maximum Sharpe ratio portfolios
 * with benchmark statistics
 * with blended statistics

## Real Data

**_Historical end-of-day financial time series data._**

The data set `http://hilpisch.com/aiif_eikon_eod_data.csv`.

## Imports and Data

In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
pd.set_option('display.precision', 4)
np.set_printoptions(suppress=True, precision=4)

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

## Example (1)

### Real Data

In [ ]:
symbols = raw.columns[:3]
symbols

In [ ]:
data = raw[symbols]

In [ ]:
rets = np.log(data / data.shift(1))

In [ ]:
noa = len(symbols)

In [ ]:
mu = rets.mean() * 252
mu

In [ ]:
cov = rets.cov() * 252
cov

### Views

#### Basics

In [ ]:
v = 1  # number of views

In [ ]:
P = pd.DataFrame(np.zeros((v, noa)), columns=symbols)
P

In [ ]:
q = np.zeros(v)

In [ ]:
omega = np.zeros((v, v))

#### Specific View

In [ ]:
P.loc[0, 'AAPL.O'] = 1

In [ ]:
P.loc[0, 'MSFT.O'] = -1

In [ ]:
q[0] = 0.02

In [ ]:
omega[0, 0] = 0.000001  # "no" variance

In [ ]:
P

In [ ]:
tau = 1

In [ ]:
C = tau * cov

### Calculations

In [ ]:
m1 = np.dot(P.T, np.dot(1 / omega, P)) + np.linalg.inv(C)

In [ ]:
m1

In [ ]:
m2 = np.dot(P.T, np.dot(1 / omega, q)) + np.dot(np.linalg.inv(C), mu)

In [ ]:
m2

In [ ]:
mu_ = np.dot(np.linalg.inv(m1), m2)

In [ ]:
mu_

In [ ]:
r = pd.DataFrame({'hist': mu, 'bl92': mu_}, index=symbols)

In [ ]:
r

In [ ]:
cov_ = np.linalg.inv(m1)

In [ ]:
cov_

In [ ]:
cov_ + cov

## Example (2)

### Views

#### Basics

In [ ]:
v = 1  # number of views

In [ ]:
P = pd.DataFrame(np.zeros((v, noa)), columns=symbols)
P

In [ ]:
q = np.zeros(v)

In [ ]:
omega = np.zeros((v, v))

#### Specific View

In [ ]:
P.loc[0, 'AAPL.O'] = 1

In [ ]:
P.loc[0, 'MSFT.O'] = -1

In [ ]:
q[0] = 0.02

In [ ]:
omega[0, 0] = 0.01  # 1% variance

In [ ]:
P

In [ ]:
C = tau * cov

### Calculations

In [ ]:
m1 = np.dot(P.T, np.dot(1 / omega, P)) + np.linalg.inv(C)

In [ ]:
# m1

In [ ]:
m2 = np.dot(P.T, np.dot(1 / omega, q)) + np.dot(np.linalg.inv(C), mu)

In [ ]:
# m2

In [ ]:
mu_ = np.dot(np.linalg.inv(m1), m2)

In [ ]:
mu_

In [ ]:
r = pd.DataFrame({'hist': mu, 'bl92': mu_}, index=symbols)

In [ ]:
r

In [ ]:
cov_ = np.linalg.inv(m1)

In [ ]:
cov_

In [ ]:
cov_ + cov

## Example (3)

### Views

#### Basics

In [ ]:
v = 2  # number of views

In [ ]:
P = pd.DataFrame(np.zeros((v, noa)), columns=symbols)
P

In [ ]:
q = np.zeros(v)
q

In [ ]:
omega = np.zeros((v, v))
omega

#### Specific View

#### View 1 

In [ ]:
P.loc[0, 'AAPL.O'] = -1  # Apple underperforms ...

In [ ]:
P.loc[0, 'MSFT.O'] = 1  # ... Microsoft ...

In [ ]:
q[0] = 0.02  # ... by 2% points

In [ ]:
omega[0, 0] = 0.01

In [ ]:
P

#### View 2 

In [ ]:
P.loc[1, 'INTC.O'] = 1  # Intel shall ...

In [ ]:
q[1] = 0.175  # ... perform much better

In [ ]:
q

In [ ]:
omega[1, 1] = 0.04

In [ ]:
omega

In [ ]:
P

In [ ]:
C = tau * cov

### Calculations

In [ ]:
m1 = np.dot(P.T, np.dot(np.linalg.inv(omega), P)) + np.linalg.inv(C)
m1

In [ ]:
# m1

In [ ]:
m2 = np.dot(P.T, np.dot(np.linalg.inv(omega), q)) + np.dot(np.linalg.inv(C), mu)
m2

In [ ]:
# m2

In [ ]:
mu_ = np.dot(np.linalg.inv(m1), m2)

In [ ]:
mu_

In [ ]:
r = pd.DataFrame({'hist': mu, 'bl92': mu_}, index=symbols)

In [ ]:
r

In [ ]:
rets.corr()

For the blended expected returns:
* `AAPL.O` under performs `MSFT.O` (view 1) &mdash; but not by as much as assumed.
* `INTC.O` performns better than previously (view 2) &mdash; but not as good as assumed.

Two major effects are at work:
* One effect is that the two views are associated with risk (positive variance), that is why e.g. `INTC.O` doesn't hit exactly the assumed level.
* The other effect is that the higher assumed expected return for `INTC.O` pushes also `APPL.O` & `MSFT.O` higher due to the positive correlation with `INTC.O`.

In [ ]:
cov

In [ ]:
cov_ = np.linalg.inv(m1)

In [ ]:
cov_

In [ ]:
cov_ + cov

## Portfolio Statistics

In [ ]:
def port_return(mu, weights):
    return np.dot(mu, weights)

In [ ]:
def port_volatility(cov, weights):
    return np.dot(weights, np.dot(cov , weights)) ** 0.5

In [ ]:
def port_sharpe(mu, cov, weights):
    return port_return(mu, weights) / port_volatility(cov, weights)

## Optimal Weights

In [ ]:
from scipy.optimize import minimize

### Historical Statistics

In [ ]:
bnds = noa * [(0, 1),]

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
opt = minimize(lambda weights: -port_sharpe(mu, cov, weights),
                  noa * [1 / noa],
                  bounds=bnds,
                  constraints=cons)

In [ ]:
opt

In [ ]:
port_return(mu, opt['x'])

In [ ]:
port_volatility(cov, opt['x'])

In [ ]:
port_sharpe(mu, cov, opt['x'])

In [ ]:
from pylab import plt

In [ ]:
plt.pie(opt['x'], labels=symbols);

### BL92 Returns

In [ ]:
opt = minimize(lambda weights: -port_sharpe(mu_, cov + cov_, weights),
                  noa * [1 / noa],
                  bounds=bnds,
                  constraints=cons)

In [ ]:
opt

In [ ]:
port_return(mu_, opt['x'])

In [ ]:
port_volatility(cov_ + cov, opt['x'])

In [ ]:
port_sharpe(mu_, cov_ + cov, opt['x'])

In [ ]:
plt.pie(opt['x'], labels=symbols);

Incorporating the two views according to BL92 leads to a significantly different portfolio composition with all three assets included and `MSFT.O` now having the largest weight (and not `AAPL.O` anymore).

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>